In [1]:
import six.moves.cPickle as Pickle
import cv2
import numpy as np

In [2]:
def loadImage(path):
    inImage_ = cv2.imread(path)
    inImage = cv2.cvtColor(inImage_, cv2.COLOR_RGB2BGR)
    info = np.iinfo(inImage.dtype)
    inImage = inImage.astype(np.float) / info.max

    iw = inImage.shape[1]
    ih = inImage.shape[0]
    if iw < ih:
        inImage = cv2.resize(inImage, (64, int(64 * ih/iw)))
    else:
        inImage = cv2.resize(inImage, (int(64 * iw / ih), 64))
    inImage = inImage[0:64, 0:64]
    return th.from_numpy(2 * inImage - 1).transpose(0, 2).transpose(
        1, 2
    )

In [3]:
class LookbookDataset():
    def __init__(self, data_dir, index_dir):
        self.data_dir = data_dir
        with open(index_dir+'cloth_table.pkl', 'rb') as cloth:
            self.cloth_table = Pickle.load(cloth)
        with open(index_dir+'model_table.pkl', 'rb') as model:
            self.model_table = Pickle.load(model)

        self.cn = len(self.cloth_table)
        self.path = data_dir

    def getbatch(self, batchsize):
        batch1 = []
        batch2 = []
        batch3 = []
        for i in range(batchsize):
            seed = th.randint(1, 100000, (1,)).item()
            th.manual_seed((i+1)*seed)
            r1 = th.randint(0, self.cn, (1,)).item()
            r2 = th.randint(0, self.cn, (1,)).item()
            r1 = int(r1)
            r2 = int(r2)
            mn = len(self.model_table[r1])
            r3 = th.randint(0, mn, (1,)).item()
            r3 = int(r3)

            path1 = self.cloth_table[r1]
            path2 = self.cloth_table[r2]
            path3 = self.model_table[r1][r3]
            img1 = loadImage(self.path + path1)
            img2 = loadImage(self.path + path2)
            img3 = loadImage(self.path + path3)
            batch1.append(img1)
            batch2.append(img2)
            batch3.append(img3)
        return th.stack(batch1), th.stack(batch2), th.stack(batch3)

In [32]:
img_size=32

In [78]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, LeakyReLU, Conv2DTranspose, BatchNormalization, ReLU, Activation
from keras import backend as K

In [85]:
class NetG():
    def __init__(self, nc=3, ngf=96):
        super(NetG, self).__init__()
        model=Sequential()
        model.add(Conv2D(ngf, input_shape=(img_size, img_size, 1), kernel_size=4, strides=2, padding=1, bias=False))
        model.add(LeakyReLU(0.2))
        
        model.add(Conv2D(ngf*2, kernel_size=4, strides=2, padding=1, bias=False))
        model.add(BatchNormalization())
        model.add(LeakyReLU(0.2))
        
        model.add(Conv2D(ngf*4, kernel_size=4, strides=2, padding=1, bias=False))
        model.add(BatchNormalization())
        model.add(LeakyReLU(0.2))
        
        model.add(Conv2D(ngf*8, kernel_size=4, strides=2, padding=1, bias=False))
        model.add(BatchNormalization())
        model.add(LeakyReLU(0.2))
        
        model.add(Conv2DTranspose(ngf*4, kernel_size=4, strides=2, padding=1, bias=False))
        model.add(BatchNormalization())
        model.add(ReLU())
        
        model.add(Conv2DTranspose(ngf*2, kernel_size=4, strides=2, padding=1, bias=False))
        model.add(BatchNormalization())
        model.add(ReLU())
        
        model.add(Conv2DTranspose(ngf, kernel_size=4, strides=2, padding=1, bias=False))
        model.add(BatchNormalization())
        model.add(ReLU())
        
        model.add(Conv2DTranspose(nc, kernel_size=4, strides=2, padding=1, bias=False))
        model.add(BatchNormalization())
        model.add(ReLU())
        
        model.add(Activation('tanh'))
        self.model=model

In [92]:
class NetD(nn.Module):
    def __init__(self, nc=3, ndf=96):
        super(NetD, self).__init__()
        
        model = Sequential()
        model.add(Conv2D(ndf, input_shape=(32, 32, 1), kernel_size=4, strides=2, padding="same", use_bias=False))
        model.add(LeakyReLU(0.2))
        
        model.add(Conv2D(ndf*2, kernel_size=4, strides=2, padding="same", use_bias=False))
        model.add(BatchNormalization())
        model.add(LeakyReLU(0.2))
        
        model.add(Conv2D(ndf*4, kernel_size=4, strides=2, padding="same", use_bias=False))
        model.add(BatchNormalization())
        model.add(LeakyReLU(0.2))
        
        model.add(Conv2D(ndf*8, kernel_size=4, strides=2, padding="same", use_bias=False))
        model.add(BatchNormalization())
        model.add(LeakyReLU(0.2))
        
        model.add(Conv2D(ndf*2, kernel_size=4, strides=2, padding="same", use_bias=False))
        model.add(BatchNormalization())
        model.add(LeakyReLU(0.2))
        
        model.add(Conv2D(1, kernel_size=4, strides=2, padding="same", use_bias=False))
        
        model.add(Activation('sigmoid'))
        self.model=model

In [93]:
class NetA(nn.Module):
    def __init__(self, nc=3, ndf=96):
        super(NetA, self).__init__()
        
        model=Sequential()
        model.add(Conv2D(ndf, input_shape=(img_size, img_size, 1), kernel_size=4, stride=2, padding=1, bias=False))
        model.add(LeakyReLU(0.2))
        
        model.add(Conv2D(ndf*2, kernel_size=4, stride=2, padding=1, bias=False))
        model.add(BatchNormalization())
        model.add(LeakyReLU(0.2))
        
        model.add(Conv2D(ndf*4, kernel_size=4, stride=2, padding=1, bias=False))
        model.add(BatchNormalization())
        model.add(LeakyReLU(0.2))
        
        model.add(Conv2D(ndf*8, kernel_size=4, stride=2, padding=1, bias=False))
        model.add(BatchNormalization())
        model.add(LeakyReLU(0.2))
        
        model.add(Conv2D(1, kernel_size=4, stride=4, bias=False))
        model.add(Activation='sigmoid')


In [94]:
netd = NetD()
a = K.zeros(shape=(128, 3, 64, 64))
# b = netd()
# print(b.shape)